In [2]:
using PyPlot
using JLD

using PDMP
using Klara

In [30]:
d = load("dex_lbps_pmf1.jld")

evl     = d["evlist"]
details = d["details"]


Dict{String,Any} with 5 entries:
  "ev_secondbranch" => 3
  "ev_firstbranch"  => 9997
  "clocktime"       => 6.71133
  "nevents"         => 0
  "globalclock"     => 1.61315e6

In [3]:
function showHist(evl::Vector{PDMP.EventList}, vi::Int64, Ns::Int64, 
                    mu=0.0, sigma=1.0, lb=-10., ub=10.)
    vi = 1
    Ns = 5000
    xs = PDMP.samplelocalpath(evl[vi], linspace(0,evl[vi].ts[end],Ns))

    PyPlot.plt[:hist](xs,Int(ceil(0.5*sqrt(Ns))),normed=true)
    
    xx = linspace(-5,5,500)
    yy = exp(-(xx-mu).^2/(2. * sigma^2))/sqrt(2pi * sigma^2)
    plot(xx,yy)
    nothing
end
;

In [13]:
details["globalclock"]

8.890000591333333e7

In [31]:
length(evl)

12

In [32]:
length(evl[1].xs)

1440

In [ ]:
# 1 to 7 = Ui
# 8 to 12 = Vj

In [33]:
T = maximum(evl[i].ts[end] for i in 1:12)

313.0892341268476

In [34]:
ss = []

for vi = 1:12

    s = evl[vi].xs[1]*0.0
    for i in 1:length(evl[vi].xs)-1
        taui = (evl[vi].ts[i+1]-evl[vi].ts[i])
        s   += evl[vi].xs[i]*taui + evl[vi].vs[i]*taui^2/2.0
    end
    s += evl[vi].xs[end]*(T - evl[vi].ts[end]) + evl[vi].vs[end]*(T - evl[vi].ts[end])^2/2.0

    push!(ss,s/T)
end

In [35]:
dot(ss[1],ss[8])

-1.7953527596802568

In [36]:
dot(ss[1],ss[9])

-1.0136249875674772

In [3]:
d = load("dex_lbps_pmfbig2.jld")

evl     = d["evlist"]
details = d["details"]


Dict{String,Any} with 5 entries:
  "ev_secondbranch" => 0
  "ev_firstbranch"  => 10000
  "clocktime"       => 1487.69
  "nevents"         => 0
  "globalclock"     => 20.2147

In [4]:
nU = 6040
nV = 3952

3952

In [5]:
length(evl) - (nU+nV)

0

In [7]:
T = maximum(evl[i].ts[end] for i in 1:12)

0.003550035476660277

In [26]:

for j = [1193, 661, 914, 3408, 2355]
    ss = []
    for vi = [1,6040+j]

        s = evl[vi].xs[1]*0.0
        for i in 1:length(evl[vi].xs)-1
            taui = (evl[vi].ts[i+1]-evl[vi].ts[i])
            s   += evl[vi].xs[i]*taui + evl[vi].vs[i]*taui^2/2.0
        end
        s += evl[vi].xs[end]*(T - evl[vi].ts[end]) + evl[vi].vs[end]*(T - evl[vi].ts[end])^2/2.0

        push!(ss,s/T)
    end
    @show dot(ss[1],ss[2])
end
@show R[1:5,:]

dot(ss[1],ss[2]) = -0.26818701112587506
dot(ss[1],ss[2]) = -0.7588134914752724
dot(ss[1],ss[2]) = -6.160943407808115
dot(ss[1],ss[2]) = 6.7412317558313815
dot(ss[1],ss[2]) = 4.3032228583971985
R[1:5,:] = [1 1193 5; 1 661 3; 1 914 3; 1 3408 4; 1 2355 5]


5×3 Array{Int64,2}:
 1  1193  5
 1   661  3
 1   914  3
 1  3408  4
 1  2355  5

In [15]:
a = readdlm("data/ratings_filtered.csv", ',', Int)
R = a[:,1:3];

In [27]:
R[500:510,:]

11×3 Array{Int64,2}:
 6  2966  5
 6   266  4
 6    17  4
 6  3699  4
 6  1441  4
 6  1088  5
 6   912  4
 6   199  5
 6   914  5
 6  3408  5
 6  1806  3

In [28]:
for j = R[500:510,2]
    ss = []
    for vi = [6,6040+j]

        s = evl[vi].xs[1]*0.0
        for i in 1:length(evl[vi].xs)-1
            taui = (evl[vi].ts[i+1]-evl[vi].ts[i])
            s   += evl[vi].xs[i]*taui + evl[vi].vs[i]*taui^2/2.0
        end
        s += evl[vi].xs[end]*(T - evl[vi].ts[end]) + evl[vi].vs[end]*(T - evl[vi].ts[end])^2/2.0

        push!(ss,s/T)
    end
    @show dot(ss[1],ss[2])
end
@show R[500:510,end]

dot(ss[1],ss[2]) = 1.119383053583336
dot(ss[1],ss[2]) = -5.914011483212761
dot(ss[1],ss[2]) = -3.982211804119346
dot(ss[1],ss[2]) = 6.770279103038423
dot(ss[1],ss[2]) = 4.61672475295694
dot(ss[1],ss[2]) = -3.8266467266875335
dot(ss[1],ss[2]) = 0.8225007484474369
dot(ss[1],ss[2]) = 8.84388005945737
dot(ss[1],ss[2]) = 11.027364805391183
dot(ss[1],ss[2]) = 3.058533786071976
dot(ss[1],ss[2]) = -8.528943942674381
R[500:510,end] = [5,4,4,4,4,5,4,5,5,5,3]


11-element Array{Int64,1}:
 5
 4
 4
 4
 4
 5
 4
 5
 5
 5
 3

In [30]:
rmse = 0.0

for k in 1:size(R,1)
    ui = R[k,1]
    vj = R[k,2] + nU
    
    su = evl[ui].xs[1]*0.0
    for i in 1:length(evl[ui].xs)-1
        taui = (evl[ui].ts[i+1]-evl[ui].ts[i])
        su  += evl[ui].xs[i]*taui + evl[ui].vs[i]*taui^2/2.0
    end
    su += evl[ui].xs[end]*(T - evl[ui].ts[end]) + evl[ui].vs[end]*(T - evl[ui].ts[end])^2/2.0
    su /= T
    
    sv = evl[vj].xs[1]*0.0
    for i in 1:length(evl[vj].xs)-1
        taui = (evl[vj].ts[i+1]-evl[vj].ts[i])
        sv  += evl[vj].xs[i]*taui + evl[vj].vs[i]*taui^2/2.0
    end
    sv += evl[vj].xs[end]*(T - evl[vj].ts[end]) + evl[vj].vs[end]*(T - evl[vj].ts[end])^2/2.0
    sv /= T
    
    rmse += (R[k,end]-dot(su,sv))^2
end

sqrt(rmse/size(R,1))

# put negative stuff to zero?

43.918102774649846

In [32]:
rmse = 0.0

for k in 1:size(R,1)
    ui = R[k,1]
    vj = R[k,2] + nU
    
    su = evl[ui].xs[1]*0.0
    for i in 1:length(evl[ui].xs)-1
        taui = (evl[ui].ts[i+1]-evl[ui].ts[i])
        su  += evl[ui].xs[i]*taui + evl[ui].vs[i]*taui^2/2.0
    end
    su += evl[ui].xs[end]*(T - evl[ui].ts[end]) + evl[ui].vs[end]*(T - evl[ui].ts[end])^2/2.0
    su /= T
    
    sv = evl[vj].xs[1]*0.0
    for i in 1:length(evl[vj].xs)-1
        taui = (evl[vj].ts[i+1]-evl[vj].ts[i])
        sv  += evl[vj].xs[i]*taui + evl[vj].vs[i]*taui^2/2.0
    end
    sv += evl[vj].xs[end]*(T - evl[vj].ts[end]) + evl[vj].vs[end]*(T - evl[vj].ts[end])^2/2.0
    sv /= T
    
    dsusv = dot(su,sv)
    dsusv = dsusv>5?5:dsusv<0?0:dsusv
    rmse += (R[k,end]-dsusv>0)^2
end

sqrt(rmse/size(R,1))

# put negative stuff to zero?

0.8621505956254085